# Importe de librerias

In [1]:
import pandas as pd
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
import os
from sklearn.model_selection import train_test_split
import torch

torch.set_float32_matmul_precision('high')

In [2]:
from dataset import LitPriceData
from trainer import LitTrainer

In [3]:
from utils import splitData

# Carga de monedas selecionadas

In [4]:
coins = pd.read_json('../data/selected_coins.json')
coins

,0
0,celestia
1,flare-networks
2,polygon-bridged-usdt-polygon
3,xdce-crowd-sale
4,injective-protocol
5,virtual-protocol
6,blockstack
7,sonic-3
8,binance-bridged-usdc-bnb-smart-chain
9,mantle-staked-ether


In [5]:
def trainModel(model, X, y, checkpoint_callback=None):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    dataModule = LitPriceData(X_train, y_train, X_val, y_val)
       
    if checkpoint_callback is not None:
        trainer = L.Trainer(max_epochs=75, accelerator="auto", callbacks=[checkpoint_callback])
    else:
        trainer = L.Trainer(max_epochs=75, accelerator="auto")

    trainer.fit(model=model, datamodule=dataModule)

In [ ]:
hyperparams = {'hidden_size': 46,
 'num_layers': 2,
 'lr': 0.006910149762728569,
 'dropout': 0.13197445494029517,
 'sequence_legth': 24}

for i, row in coins.iterrows():    
    name = row[0]
    export_dir = f"../models/exports/{name}"
    os.makedirs(export_dir, exist_ok=True)

    checkpoint_callback = ModelCheckpoint(
        dirpath = f"../models/checkpoints/{name}",
        filename = "checkpoints",
        save_top_k=1,
        monitor="val_loss",
        mode="min"
    )

    #Change the values
    model = LitTrainer(hidden_size=hyperparams["hidden_size"], num_layers=hyperparams["num_layers"], lr=hyperparams["lr"], dropout=hyperparams["dropout"])
    X, y = splitData(name, sequence_length=hyperparams["sequence_legth"])
    trainModel(model, X, y, checkpoint_callback)
    
    best_ckpt_path = checkpoint_callback.best_model_path
    print(f"Best checkpoint for {name}: {best_ckpt_path}")

    best_model = LitTrainer.load_from_checkpoint(best_ckpt_path, hidden_size=hyperparams["hidden_size"], num_layers=hyperparams["num_layers"], dropout=hyperparams["dropout"])

    export_state_dict_path = os.path.join(export_dir, "best_model.pth")
    torch.save(best_model.state_dict(), export_state_dict_path)
